In [2]:
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
from hello.pid.lvpid import PIDController
from hello.pid.delay import m2s, s2m, h2s, seconds, minutes, hours, days, DelayBuffer, DelaySink

Using matplotlib backend: TkAgg


In [25]:
class TempProcess():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.tdelay = DelayBuffer(delay, initial).cycle
        self.g = g / 60
        self.k = k / 60
        self.env = env
        
    def step(self, pv, op):
        op = self.tdelay(op)
        dT = pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        return pv + dpv
    
    
class TempProcess2():
    def __init__(self, delay, initial=20, env=18.5, g=0.0019254, k=-0.001579031):
        """
        :param g: gain in units of C/min/%
        :param k: decay rate in units of C/min/dT
        """
        self.heatsink = DelaySink(delay, initial, 0.95)
        self.g = g / 60
        self.k = k / 60
        self.env = env
        self.pv = initial
        
    def step(self, op):
        op = self.heatsink.delay(op)
        dT = self.pv - self.env
        decay = self.k * dT
        gain = self.g * op
        dpv = decay + gain
        self.pv += dpv
        return self.pv

In [26]:
def paste(cells, data, offset=0):
    with screen_lock(xl):
        cells.Range(cells(1,1+offset), cells(1, 4+offset)).Value = [("T", "PV", "OP", "DPV")]
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()
        cells.Range(topleft, bottomright).Value = data
    
def clear(cells, data, offset=0):
    if not data: return
    with screen_lock(xl):
        topleft = cells(2,1+offset)
        bottomright = topleft.Offset(len(data), len(data[0]))
        cells.Range(topleft, bottomright).Clear()

In [27]:
from officelib.xllib import *
xl = Excel()

try:
    wb
except NameError:
    wb = xl.Workbooks.Add()

try:
    ws = wb.Worksheets(1)
    ws2 = wb.Worksheets(2)
except Exception:
    wb = xl.Workbooks.Add()
    ws = wb.Worksheets.Add()
    ws2 = wb.Worksheets.Add()

cells = ws.Cells
cell_range = ws.Cells.Range

In [38]:
def temp_sim(ops):

    # unpack (lazy refactor)
    pv = ops.initial_pv
    sp = ops.setpoint
    op = ops.initial_output
    mode = ops.mode
    k = ops.k
    g = ops.g
    delay = ops.delay
    end = ops.end
    
    pid = PIDController(pgain=ops.p, itime=m2s(ops.i), dtime=m2s(ops.d), auto_max=ops.auto_max,
                        auto_min=ops.auto_min, beta=ops.beta, linearity=ops.linearity, alpha=ops.alpha,
                        deadband=0, sp_high=100, sp_low=0, gamma=ops.gamma)

    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    elif mode == 'o2a':
        pid.off_to_auto(pv, sp)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = ops.process(delay, op, env=ops.env, g=g, k=k)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t/60, pv, op))
        if t >= end:
            break
    t, pv, op = list(zip(*data))
    dpvdt = np.gradient((np.gradient(pv, 1/60)), 1/60) ** 2
    #dpvdt = np.abs(np.gradient(pv, 1/3600))
    data = list(zip(t, pv, op, dpvdt))
    return data

def temp_sim2(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37, 
             g=0.0019254, k=-0.001579031, mode='m2a', beta=1, Klass2=TempProcess, 
              decay=.002, decay_time=240, end2=120):
    pid = PIDController(p,m2s(i),m2s(d),amax,amin, do_ifactor=True, beta=beta)
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = Klass2(delay, op, g=g, k=k)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end:
            break
    for _ in range(decay_time):
        t += 1
        op = pid.step(pv, sp)
        pv -= decay
        proc.step(pv, op)
        data.append((t, pv, op))
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end + end2 +decay_time:
            break
    return data
    
def temp_sim3(p, i, d, delay, amax, amin, end, op=0, pv=20, sp=37, 
             g=0.0019254, k=-0.001579031, mode='m2a', beta=1, pv_delay=240):
    pid = PIDController(p,m2s(i),m2s(d),amax,amin, do_ifactor=True, beta=beta)
    if mode == 'm2a':
        pid.man_to_auto(pv, sp, op)
    else:
        pid.man_to_auto(pv, pv, op)
    proc = TempProcess3(delay, op, g=g, k=k, pv_delay=pv_delay, initial_pv=pv)
    t = 0
    data = [(t, pv, op)]
    while True:
        t += 1
        op = pid.step(pv, sp)
        pv = proc.step(pv, op)
        data.append((t, pv, op))
        if t >= end:
            break
    return data
    

In [39]:
from pysrc.snippets import OptionCategory

offset = 0
p = 60
i = 35
data = []
delay=m2s(15)

class TempOps(OptionCategory):
    # PID 
    p = 60
    i = 35
    d = 0
    alpha = -1
    linearity = 1
    beta = 1
    gamma = 0
    auto_max = 100
    auto_min = 0
    
    # plant    
    k = -0.00357  # C / min * C
    g = 0.002053  # C / min * % 
    delay = m2s(5)
    
    # Simulation
    initial_pv = 20
    env = 20
    initial_output = 0
    setpoint = 37
    end = 5 * hours
    mode = 'o2a'
    process = TempProcess2
    

In [40]:
def declare_list(name):
    if name not in globals():
        globals()[name] = []
        
declare_list("data")
declare_list("a2a_data")
declare_list("a2a_data2")

In [41]:
from time import sleep
clear(cells, data)
ws2 = wb.Worksheets("Sheet2")
cells2 = ws2.Cells
cells2.Range("E:I").Clear()
cells2.Range("E2:I2").Value = [("P", "I", "Max T", "Stable Time", "Stable")]
n = 0
current = cells2.Range("E3")

def time_stable(data, at, margin):
    rev = reversed(data)
    hi = at + margin
    lo = at - margin
    time_stable = data[-1][0] - \
            next((t for t, pv, _ in reversed(data) if not (lo <= pv <= hi)), 0)
    return time_stable

def max_t(data):
    max_pv = 0
    max_time = 0
    for t, pv, _ in data:
        if pv > max_pv:
            max_pv = pv
            max_time = t
    return max_pv, max_time

combos = []
for p in range(10, 101, 10):
    for i in range(10, 101, 10):
        combos.append((p, i))
def run_test():
    global data, stable, current
    with HiddenXl(xl):
        for n, (p, i) in enumerate(combos, 1):
            print("\rGenerating data for P:%.2f I:%.2f (Test %d of %d)     " % (p, i, n, len(combos)), end="")
            data = temp_sim2(p=p, i=i, d=0, delay=delay, 
                    amax=100, amin=0, end=m2s(60), op=15.17, 
                    pv=37, sp=37, mode='a2a', beta=0, Klass2=TempProcess2, decay=.002, decay_time=900, end2=m2s(150))
            cells.Range("E1:F1").Value = [(p, i)]
            paste(cells, data, 0)
            current.Value = "%.2f" % p
            current.Offset(1, 2).Value = "%.2f" % i
            current.Offset(1, 3).Value = xl.WorksheetFunction.Max(cells.Range("B:B"))
            stable = time_stable(data, 37, 0.05)
            current.Offset(1, 4).Value = stable
            current.Offset(1, 5).Value = stable > 3600
            current = current.Offset(2, 1)

In [42]:
def summarize(cells, data, ops, margin, n):
    header = [("PGain", "Itime", "DTime", "Mode", 
               "Start", "Setpoint", 
               "Settle Time", "Max", "Sum Derivative")]
    if not cells(1,1).Value:
        cells.Range(cells(1,1), cells(1, len(header[0]))).Value = header
    rev = reversed(data)
    hi = ops.setpoint + margin
    lo = ops.setpoint - margin
    settle_time = next((t for t, pv, *_ in rev if not (lo <= pv <= hi)), 0)
    max_temp = max(pv for _, pv, *_ in data)
    sd = sum(dpvs for t,_,_,dpvs in data if 1 < t < 2) / 3600
    target_range = cells.Range(cells(1+n, 1), cells(1+n, len(header[0])))
    target_range.Value = [(ops.p, ops.i, ops.d, ops.mode, 
                           ops.initial_pv, ops.setpoint, 
                           settle_time, max_temp, sd)]

In [44]:
ops = TempOps()

# PID
ops.p = 70
ops.i = 20
ops.d = 0
ops.alpha = -1
ops.linearity = 1
ops.beta = 0
ops.gamma = 0
ops.auto_max = 50
ops.auto_min = 0

# plant    
ops.k = -0.00357 * 1    # C / min * C
ops.g = 0.00205296  * 2.1 # C / min * % 
ops.delay = m2s(8)

# Simulation
ops.initial_pv = 24
ops.env = 21.972
ops.initial_output = 0
ops.setpoint = 37
ops.end = 2 * hours
ops.mode = 'o2a'
ops.process = TempProcess2
n = 0
#clear_data()
dbgop = [0]
def _dbg(op):
    dbgop.append(op)
with screen_lock(xl):
    data = temp_sim(ops)
    paste(cells, data, 0)
    summarize(ws2.Cells, data, ops, 0.05, n+1)

#     ops.mode = 'a2a'
#     ops.initial_pv = 35
#     ops.initial_output = 11.6
#     a2a_data = temp_sim(ops)
#     paste(cells, a2a_data, 6)
#     summarize(ws2.Cells, a2a_data, ops, 0.05, n+2)

#     ops.initial_pv = 35.8
#     a2a_data2 = temp_sim(ops)
#     paste(cells, a2a_data2, 12)
#     summarize(ws2.Cells, a2a_data2, ops, 0.05, n+3)
    
# reset initial pv so below cells don't fuck up
ops.initial_pv = 24
t, pv, op, dpv = list(zip(*data))
from matplotlib import pyplot as plt
import numpy as np
%matplotlib
plt.close()
print(len(t), len(op), len(dbgop))
plt.plot(t, dbgop, "blue")
plt.plot(t, op, "red")
sum(op)/len(op), sum(dbgop)/len(dbgop)

Using matplotlib backend: TkAgg
7201 7201 7201


(34.25295066823022, 33.95788182913609)

Using matplotlib backend: TkAgg
18001 18001 18001


(21.21705792214463, 21.05125214690666)

In [ ]:
def clear_data(ws=ws):
    with screen_lock(xl):
        for c in ws.UsedRange.Columns:
            if c.Column not in range(5,14):
                c.EntireColumn.Clear()

def compare_values(values, param):
    global chart, chart2, data
    with screen_lock(xl):
        clear_data()
        c = 1
        chart = wb.Charts(2)
        chart2 = wb.Charts(3)
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        xr = cells.Range("H8:H1106")
        yr = cells.Range("F8:F1106")
        yr2 = cells.Range("G8:G1106")
        CreateDataSeries(chart, xr, yr, "Actual")
        CreateDataSeries(chart2, xr, yr2, "Actual")

        for v in values:
            setattr(ops, param, v)
            data = temp_sim(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "%s=%.2g"%(param, v))
            CreateDataSeries(chart2, xr, yr2, "%s=%.2g"%(param,v))
            c += 5
            if c == 6:
                c = 10
            
def compare_values2(values, param, base, label=None):
    global chart, chart2, data
    if label is None:
        label = param
    with screen_lock(xl):
        clear_data()
        c = 1
        chart = wb.Charts(2)
        chart2 = wb.Charts(3)
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        xr = cells.Range("H8:H1106")
        yr = cells.Range("F8:F1106")
        yr2 = cells.Range("G8:G1106")
        CreateDataSeries(chart, xr, yr, "Actual")
        CreateDataSeries(chart2, xr, yr2, "Actual")

        for v in values:
            setattr(ops, param, v * base)
            data = temp_sim(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "%s=%.3g"%(label, v))
            CreateDataSeries(chart2, xr, yr2, "%s=%.3g"%(label,v))
            c += 5
            if c == 6:
                c = 10
                

def compare_values3(args, labels):
    global chart, chart2, data
    with screen_lock(xl):
        clear_data()
        c = 1
        chart = wb.Charts(2)
        chart2 = wb.Charts(3)
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        
        xr = cells.Range("H8:H1106")
        yr = cells.Range("F8:F1106")
        yr2 = cells.Range("G8:G1106")
        CreateDataSeries(chart, xr, yr, "Actual")
        CreateDataSeries(chart2, xr, yr2, "Actual")
        
        values, params, bases, operations = list(zip(*args))
        for *vs, l in zip(*values, labels):
            for v, p, base, op in zip(vs, params, bases, operations):
                if op == "*":
                    val = base * v
                elif op == "=":
                    val = v
                elif op == "+":
                    val = base + v
                elif op == "-":
                    val = base - v
                elif op == "--":
                    val = v - base
                elif op == "/":
                    val = base / v
                elif op == "\\":
                    val = v / base
                else:
                    raise ValueError(op)
                setattr(ops, p, base * v)
            data = temp_sim(ops)
            paste(cells, data[::15], c-1)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            yr2 = cells.Range(cells(2,c+2), cells(2,c+2).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, l)
            CreateDataSeries(chart2, xr, yr2, l)
            
            c += 4
            if c == 5:
                c = 10
    

In [257]:
k = -0.00357  # C / min * C
g = 0.00205296  * 1.05 # C / min * % 
ops.auto_max = 50
kms = [i/100 for i in range(110, 150, 20)]
gms = [m2s(i) for i in range(5, 16, 5)]
op = "*"

combos = [(_k, _g) for _k in kms for _g in gms]
kms, gms = list (zip(*combos))

gms = [g*2 for g in gms]

labels = ["km = %.2f d = %.2f" % (k, g/60) for k, g in zip(kms, gms)]

args = [(kms, 'k', k, op), (gms, 'd', 0, "=")]
compare_values3(args, labels)

In [201]:
# impact of process decay constant multiplier on heat ramp
values = [(1+v/100) for v in range(-40, 1, 20)]
compare_values2(values, "k", -0.00357, 'Km')

In [241]:
# impact of process gain constant multiplier on heat ramp
values = [(1+cm/100) for cm in range(80, 111, 20)]
compare_values2(values, "g", 0.002053, "gm")

In [94]:
ops.delay=m2s(7)
values = [g/100 for g in (0.178437, 0.205296, 0.151579)]
compare_values(values, 'g')

In [242]:
# impact of process gain constant multiplier on heat ramp
values = [(1-cm/100) for cm in range(-10, 11, 5)]
values = (2.1,)
compare_values2(values, "g", 0.002053, "gm")

In [394]:
def compare_values4(combos):
    global chart, chart2, data
    ws3 = wb.Worksheets("Sheet3")
    cells = ws3.Cells
    with screen_lock(xl):
        ws3.UsedRange.Clear()
        c = 1
        chart = ws3.ChartObjects(1).Chart
        chart2 = ws3.ChartObjects(2).Chart
        PurgeSeriesCollection(chart)
        PurgeSeriesCollection(chart2)
        ntests = 2 * len(combos)
        n = 1
        for cb in combos:
            p, i = cb
            ops.p = p
            ops.i = i
            ops.mode = 'o2a'
            ops.initial_pv = 24
            print("\rRunning test %d of %d" % (n, ntests), end="")
            data = temp_sim(ops)
            paste(ws3.Cells, data[::15], c-1)
            summarize(cells2, data, ops, 0.05, n)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            CreateDataSeries(chart, xr, yr, "P:%d I:%d"%(p,i))
            
            n += 1
            c += 6

            ops.mode = 'a2a'
            ops.initial_pv = 35
            data = temp_sim(ops)
            paste(ws3.Cells, data[::15], c-1)
            summarize(cells2, data, ops, 0.05, n)
            xr = cells.Range(cells(2, c), cells(2,c).End(xlc.xlDown))
            yr = cells.Range(cells(2,c+1), cells(2,c+1).End(xlc.xlDown))
            CreateDataSeries(chart2, xr, yr, "P:%d I:%d"%(p,i))
            
            n += 1
            c += 6
        print("\rDone                                           ")

In [395]:
cells2 = ws2.Cells
ws2.UsedRange.Clear()
n = 1
combos = [(p, i) for p in range(60, 71, 10) for i in range(10, 26, 5)]

compare_values4(combos)

Done                                           


In [399]:
clear(cells, data, 5)
data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(300), op=13.5317397598376, 
                pv=pv, sp=37, mode='a2a', beta=1, Klass2=TempProcess)
paste(cells, data, 5)

In [404]:
clear(cells, data, 0)
data = temp_sim(p=60, i=20, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(300), op=0, 
                pv=23, sp=37, g=0.0019254, k=-0.001579031, mode='o2a', beta=1)
paste(cells, data, 0)

In [477]:
clear(cells, data, 0)
data = temp_sim3(p=60, i=20, d=0, delay=m2s(4), 
                amax=100, amin=0, end=m2s(300), op=0, 
                pv=23, sp=37, g=0.0019254, k=-0.001579031, 
                 mode='o2a', beta=1, pv_delay=1)
paste(cells, data, 0)

0.000156409306339
-0.000234696226035
0.000467907482415
-0.00101132195881
0.00186488098035
-0.00395047087269
0.0075815892458
-0.0155071243582
0.030501340191
-0.0613930452101
0.121941825917
-0.244024788118
0.486314122645
-0.971353318651
1.93783936854
-3.86835048684
7.71956841043
-15.40749059
30.7491562675
-61.3696437579
122.479671541
-244.444062048
487.856762174
-973.658153679
1943.21111063
-3878.23228331
7740.11595012
-15447.6068731
30830.0978101
-61530.2416565
122801.119628
-245084.608725
489136.134631
-976210.461978
1948305.99652
-3888399.48726
7760408.57642
-15488105.4494
30910925.3745
-61691554.8963
123123067.306
-245727145.777
490418501.521
-978769789.047
1953413863.83
-3898593689.86
7780754012.26
-15528710559.6
30991964437.4
-61853291424.5
123445858612.0
-246371367755.0
491704229951.0
-981335826299.0
1.95853512196e+12
-3.90881461898e+12
7.8011527872e+12
-1.55694221245e+13
3.10732159596e+13
-6.20154519772e+13
1.23769496177e+14
-2.47017278688e+14
4.92993329171e+14
-9.83908590921e+14

c:\program files\python35\lib\site-packages\ipykernel\__main__.py:60: RuntimeWarning: overflow encountered in double_scalars
c:\program files\python35\lib\site-packages\ipykernel\__main__.py:71: RuntimeWarning: overflow encountered in double_scalars
c:\program files\python35\lib\site-packages\ipykernel\__main__.py:88: RuntimeWarning: overflow encountered in double_scalars
c:\program files\python35\lib\site-packages\ipykernel\__main__.py:90: RuntimeWarning: invalid value encountered in double_scalars


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [239]:
clear(cells, data, 0)
data = temp_sim(p=p, i=i, d=0, delay=delay, 
                amax=100, amin=0, end=m2s(300), op=37, 
                pv=35.8, sp=37, mode='a2a', beta=0)
paste(cells, data)

In [150]:
from time import sleep
sp = 37
pv = 35.8
op = 13.5317397598376
clear(cells, data)
cells.Range("E2:F2").Value = [("Beta", "Max T")]
n = 0
current = cells.Range("E3")
for beta in range(1, 11):
    beta /= 10
    data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                amax=100, amin=0, end=m2s(600), op=op, 
                pv=pv, sp=sp, g=0.0019254, k=-0.001579031, mode='a2a', beta=beta)
    paste(cells, data, 0)
    current.Value = "%.2f" % beta
    current.Offset(1, 2).Value = xl.WorksheetFunction.Max(cells.Range("B:B"))
    current = current.Offset(2, 1)
    
    

In [209]:
n = 0
highest = 36
highest_n = 0
with screen_lock(xl):
    while True:
        print("\rTesting with Start=%.1f" % (35+n), end="")
        clear(cells, data, 0)
        data = temp_sim(p=p, i=i, d=0, delay=m2s(4.7), 
                        amax=100, amin=0, end=m2s(300), op=13.5317397598376, 
                        pv=35+n, sp=37, mode='a2a')
        paste(cells, data)
        v = cells.Range("D1").Value
        if v > highest:
            highest = v
            highest_n = n
        if n > 2:
            break
        n += 0.1
print()
print("highest", highest)
print("start", highest_n+35)

 Testing with Start=37.0
highest 37.350761910615724
start 35.8
